In [4]:
#%pip install cartopy

In [5]:
import numpy as np
import pandas as pd
import random
from Graphs.RandomGraph import RandomGraph
from Graphs.MapGraph import MapGraph
random.seed(0)
np,random.seed(0)

        
cities = [
    "Boos", "Rouen", "Franqueville-Saint-Pierre", "Le Mesnil-Esnard", "Saint-Etienne-Du-Rouvray",
    "Montmain", "Bois-Guillaume", "Bonsecours", "Sotteville-lès-Rouen", "Mont-Saint-Aignan",
    "Le Grand-Quevilly", "Le Petit-Quevilly", "Canteleu", "Déville-lès-Rouen", "Darnétal",
    "Elbeuf", "Oissel", "Maromme", "Cléon", "Saint-Aubin-lès-Elbeuf", "Saint-Aubin-Celloville",
    "Saint-Martin-du-Vivier", "Saint-Jacques-sur-Darnétal", "Saint-Léger-du-Bourg-Denis",
    "Saint-Pierre-de-Manneville", "Saint-Pierre-de-Varengeville", "Saint-Martin-de-Boscherville",
    "Saint-Aubin-Épinay", "Saint-Paër", "Isneauville", "Hénouville", "Houppeville", "Malaunay",
    "Jumièges", "La Bouille", "Duclair", "Notre-Dame-de-Bondeville", "Petit-Couronne"
]
# cities = ["Rouen", "Saint-Etienne-Du-Rouvray","Mont-Saint-Aignan", "Bonsecours"]

# cities = ["Bouffry","Yermenonville","Valennes","Mondoubleau","Arquenay", "Nantes", "Boos", "La Rochelle", "Bordeaux", "Toulouse", "Montpellier", "Marseille", "Lyon", "Le Mans", "Riom","Bergerac","Vic-sur-Cère", "Clermont-Ferrand","Sucé-sur-Erdre", "Rezé","Rieux"]
#cities = ["Marseille", "Rouen", "Lille", "Quimper", "Colmar", "Paris", "Bayonne", "Toulouse"]
#create real graph
# graph = MapGraph(cities=cities,start_city="Rouen")
graph = RandomGraph(node_number=200,min_weight=1,max_weight=100)
# graph.plot_graph()

KeyboardInterrupt: 

In [ ]:
#Filtration
edge_blocking_percentage = 50
edge_costly_percentage = 20
graph.apply_edge_blocking(percentage=edge_blocking_percentage)
graph.apply_edge_costly(percentage=edge_costly_percentage)

# if(edge_blocking_percentage > 0 or edge_costly_percentage > 0):
#     graph.plot_graph()

Removed 9850 edges of 19900
Costed 2010 edges of 10050


In [ ]:
import datetime
from Algorithms.ACO.AcoAlgo import AcoAlgo
from Algorithms.Greedy.GreedyAlgo import GreedyAlgo
from Algorithms.Tabu.TabuAlgo import TabuAlgo
from Algorithms.Genetic.GeneticAlgo import GeneticAlgo
from utils.plot import Plot

def run_algorithm(graph,algorithm,plot_path = True):
    algorithm.run()
    print(f"Algorithm {algorithm.name} : \n")
    vehicles_paths = {i + 1: path for i, path in enumerate(algorithm.paths)}
    current_path_results = {}
    algorithms_path_results = {}
    current_path_results[algorithm.name] = vehicles_paths
    algorithms_path_results[algorithm.name] = current_path_results[algorithm.name]
    
    # Print the best path found and the total distance
    print(f"Execution time : {algorithm.execution_time} s ({algorithm.execution_time * 1000} ms)")
    print(f"Best paths: {algorithm.paths}")
    print(f"Total distance: { algorithm.distance}")
    print(f"Distance per vehicles  : {algorithm.distance_per_vehicles}")
    print(f"Average distance for each vehicles : {algorithm.distance_average_per_vehicles}")
    print(f"Standard deviation distance for each vehicles : {algorithm.distance_standard_deviation_per_vehicles}")
    print(f"Number of iterations needed to find the solution : {algorithm.iterations_needed}")
    print(f"Total number of iterations : {algorithm.total_interations_realized}")
    if plot_path:
        graph.plot_graph(current_path_results)

def time_over_iterations_comparison(graph,algorithms, max_iterations, step = 1, single_plot = True, full_plot = True):
    time_results = {algo.name: [] for algo in algorithms}
    iteration_range = range(1, max_iterations + 1, step)
    for algorithm in algorithms:
        algorithm.convergence_threshold = 0
        for i in iteration_range:
            algorithm.max_iterations = i
            algorithm.min_iterations = i
            algorithm.run()
            time_results[algorithm.name].append(algorithm.execution_time)
            print(f"{algorithm.name} - Iterations: {i} of {max_iterations}, Time: {algorithm.execution_time}s")
        if single_plot:
            single_time_result = {}
            single_time_result[algorithm.name] = time_results[algorithm.name]
            Plot.plot_time_over_iterations_comparison(single_time_result,step)
    if full_plot:
        Plot.plot_time_over_iterations_comparison(time_results,step)

def distance_over_iterations_comparison(graph,algorithms, single_plot = True, full_plot = True, plot_path_single = True, full_path_plot = False):
    algorithms_distance_results = {}
    for algorithm in algorithms:
        algorithm.run()
        algorithms_distance_results[algorithm.name] = algorithm.distance_history
        if single_plot:
            single_distance_results = {}
            single_distance_results[algorithm.name] = algorithm.distance_history
            Plot.plot_distance_over_iterations_comparison(single_distance_results)
    if full_plot:
        Plot.plot_distance_over_iterations_comparison(algorithms_distance_results)

def average_algorithms_values(graph,algorithms, runs = 10):
    results = {}

    for algorithm in algorithms:
        print(f"Running {algorithm.name} for {runs} runs...")
        for run in range(runs):
            algorithm.run()
            results[algorithm.name].append(algorithm.distance)
            print(f"Run {run + 1}/{runs} for {algorithm.name}: Best distance = {algorithm.distance}")

    final_results = {}
    for algo_name, distances in results.items():
        final_results[algo_name] = {
            "best": [min(distances[:i + 1]) for i in range(len(distances))],
            "worst": [max(distances[:i + 1]) for i in range(len(distances))],
            "average": [sum(distances[:i + 1]) / (i + 1) for i in range(len(distances))]
        }

    # Appeler la méthode de traçage
    Plot.plot_average_algorithms_values(final_results)


def get_best_aco_parameters(graph = None, nodes_number = 10,threshold = 50, test_iterations = 10, num_vehicles = 1,execute_best_for_comparison = False,best_decay = 0.3, best_ants=500, best_alpha = 1.5, best_beta = 2):
    if graph is None:
        graph = RandomGraph(node_number=nodes_number,min_weight=1,max_weight=100)
    
    edge_blocking_percentage = 10
    edge_costly_percentage = 20
    graph.apply_edge_blocking(percentage=edge_blocking_percentage)
    graph.apply_edge_costly(percentage=edge_costly_percentage)
    
    best_distance = np.inf
    best_execution_time = np.inf
    best_distance_standard_deviation_per_vehicles = np.inf
    results = []
    best_params = None

    if execute_best_for_comparison:
        ACO = AcoAlgo(graph=graph,name="ACO",num_vehicles = num_vehicles,
                    num_ants=best_ants,decay=best_decay,alpha=best_alpha,beta=best_beta,
                    min_iterations=0,max_iterations=500,convergence_threshold=threshold)
        ACO.run()
        result = {
            "num_ants": best_ants,
            "decay": best_decay,
            "alpha": best_alpha,
            "beta": best_beta,
            "distance": ACO.distance,
            "execution time":ACO.execution_time,
            "standard deviation": ACO.distance_standard_deviation_per_vehicles,
            "iteration needed": ACO.iterations_needed,
            "total iterations": ACO.total_interations_realized
        }
        results.append(result)
        print(f"Run {0}/{test_iterations}: {result}")

    for i in range (1,test_iterations + 1):
        num_ants = random.randint(57,500)
        decay = random.uniform(0.24,7.3)
        alpha = random.uniform(0.33,2.93)
        beta =  random.uniform(0.91,2.94)
        
        ACO = AcoAlgo(graph=graph,name="ACO",num_vehicles = num_vehicles,
                      num_ants=num_ants,decay=decay,alpha=alpha,beta=beta,
                      min_iterations=0,max_iterations=100,convergence_threshold=threshold)
        ACO.run()
        result = {
            "num_ants": num_ants,
            "decay": decay,
            "alpha": alpha,
            "beta": beta,
            "distance": ACO.distance,
            "execution time":ACO.execution_time,
            "standard deviation": ACO.distance_standard_deviation_per_vehicles,
            "iteration needed": ACO.iterations_needed,
            "total iterations": ACO.total_interations_realized
        }
        

        results.append(result)

        if ACO.distance < best_distance and ACO.execution_time <= best_execution_time and ACO.distance_standard_deviation_per_vehicles <= best_distance_standard_deviation_per_vehicles:
            best_distance = ACO.distance
            best_params = result

        print(f"Run {i}/{test_iterations}: {result}")

    if ACO.distance < best_distance and ACO.execution_time <= best_execution_time and ACO.distance_standard_deviation_per_vehicles <= best_distance_standard_deviation_per_vehicles:
        best_distance = ACO.distance
        best_params = result

    results_df = pd.DataFrame(results)
    print("Best Paramters:\n")
    print(best_params)
    print("Datas : \n")
    print(results_df)

    now = datetime.datetime.now()
    results_df.to_csv(f"aco_results_{now.day}-{now.hour}:{now.minute}_for_{nodes_number}_nodes_and_{test_iterations}.csv",index=False)

# def run_algorithms(graph, algorithms):
#     for algorithm in algorithms:
#         run_algorithm(graph,algorithm)
#     Plot.plot_distance_over_iterations_comparison(algorithms_distance_results)
#     graph.plot_graph(algorithms_path_results)
min_iterations = 200
max_iterations = 200
convergence_threshold = 0
num_vehicles = 1
ACO = AcoAlgo(graph,"ACO",num_vehicles,100,0.5,1,2,min_iterations,max_iterations,convergence_threshold)
GREEDY = GreedyAlgo(graph,"GREEDY",num_vehicles,min_iterations,max_iterations,convergence_threshold)
TABU = TabuAlgo(graph,"TABU",num_vehicles,10,min_iterations,max_iterations,convergence_threshold)
GENETIC = GeneticAlgo(graph, "GENETIC",num_vehicles,20,min_iterations=min_iterations, max_iterations=max_iterations, convergence_threshold=convergence_threshold)
algorithms = [GREEDY]
# run_algorithms(graph= graph,algorithms=algorithms)
# graph = RandomGraph(node_number=10,min_weight=1,max_weight=100)
# time_over_iterations_comparison(graph,algorithms,max_iterations=1000, step=100)
#distance_over_iterations_comparison(graph,algorithms)


In [ ]:
node_number = 20
test_iteration = 50
graph = RandomGraph(node_number=node_number,min_weight=1,max_weight=10)

On cherche les valeurs en prenant ... avec un graphe aleatoire a chaque fois

In [ ]:
import numpy as np
import pandas as pd
import random

def random_search(cycles, node_sizes, num_trials=100):
    best_params = None
    best_distance = float('inf')

    for trial in range(num_trials):
        decay = random.uniform(0.1, 1.0)
        alpha = random.uniform(0.1, 5.0)
        beta = random.uniform(0.1, 5.0)

        print(f"Testing trial {trial + 1}/{num_trials} with decay={decay}, alpha={alpha}, beta={beta}")

        results = []
        for node_number in node_sizes:
            graph = RandomGraph(node_number=node_number, min_weight=1, max_weight=100)
            edge_blocking_percentage = random.randint(0, 20)
            edge_costly_percentage = random.randint(0, 20)
            graph.apply_edge_blocking(percentage=edge_blocking_percentage)
            graph.apply_edge_costly(percentage=edge_costly_percentage)

            aco = AcoAlgo(
                graph=graph,
                name="ACO",
                num_ants=100,
                num_vehicles=random.randint(1, 3),
                decay=decay,
                alpha=alpha,
                beta=beta,
                min_iterations=0,
                max_iterations=500,
                convergence_threshold=10,
            )
            aco.run()

            results.append({
                "node_number": node_number,
                "decay": decay,
                "alpha": alpha,
                "beta": beta,
                "distance": aco.distance,
                "std_dev": aco.distance_standard_deviation_per_vehicles,
            })

        results_df = pd.DataFrame(results)

        avg_distance = results_df["distance"].mean()
        if avg_distance < best_distance:
            best_distance = avg_distance
            best_params = (decay, alpha, beta)

    print(f"Meilleure combinaison trouvée : decay={best_params[0]}, alpha={best_params[1]}, beta={best_params[2]}")
    return best_params

node_sizes = [random.randint(10, 50) for _ in range(20)] 
best_params = random_search(cycles=5, node_sizes=node_sizes)


Testing trial 1/100 with decay=0.18225215373877868, alpha=1.3018375548316394, beta=4.609371678685776
Removed 120 edges of 903
Costed 101 edges of 783
Removed 74 edges of 496
Costed 37 edges of 422
Costed 3 edges of 66
Removed 14 edges of 378
Costed 69 edges of 364
Removed 11 edges of 91
Costed 1 edges of 80
Removed 35 edges of 231
Costed 9 edges of 196
Removed 93 edges of 820
Costed 65 edges of 727
Costed 82 edges of 1176
Removed 34 edges of 253
Costed 6 edges of 219
Removed 75 edges of 1128
Costed 189 edges of 1053
Removed 56 edges of 325
Costed 18 edges of 269
Removed 20 edges of 325
Costed 18 edges of 305
Removed 9 edges of 120
Costed 5 edges of 111
Removed 42 edges of 378
Costed 50 edges of 336
Removed 124 edges of 820
Costed 76 edges of 696
Removed 43 edges of 253
Costed 29 edges of 210
Removed 2 edges of 55
Costed 4 edges of 53
Removed 122 edges of 861
Costed 73 edges of 739
Removed 24 edges of 210
Costed 3 edges of 186
Removed 9 edges of 136
Costed 12 edges of 127
Testing trial 

KeyboardInterrupt: 

On cherche la meilleure combinaison avec un graphe aleatoire a chaque fois 

In [1]:
import numpy as np
import random
import pandas as pd
from Algorithms.ACO.AcoAlgo import AcoAlgo
from Graphs.RandomGraph import RandomGraph

# Paramètres d'exploration
alphas = np.linspace(0.1, 5.0, 10)
betas = np.linspace(0.1, 5.0, 10)
decays = np.linspace(0.1, 1.0, 10)

# Calcul du total d'itérations
total_iterations = len(alphas) * len(betas) * len(decays)
current_iteration = 0

# Stockage des résultats
results = []

# Exploration des combinaisons
def explore_parameters(graph = None):
    generate_graph = (graph == None)
    
    global current_iteration
    for decay in decays:
        for alpha in alphas:
            for beta in betas:
                current_iteration += 1
                if generate_graph:
                    graph = RandomGraph(node_number=15, min_weight=1, max_weight=100)
                graph.apply_edge_blocking(percentage=random.randint(0, 20))
                graph.apply_edge_costly(percentage=random.randint(0, 20))

                aco = AcoAlgo(
                    graph=graph,
                    name="ACO",
                    num_ants=100,
                    num_vehicles=3,
                    decay=decay,
                    alpha=alpha,
                    beta=beta,
                    min_iterations=0,
                    max_iterations=500,
                    convergence_threshold=10,
                )
                aco.run()

                distance = aco.distance
                results.append({
                    "alpha": alpha,
                    "beta": beta,
                    "decay": decay,
                    "distance": distance
                })

def show_best_parameters(top_n=5):
    df = pd.DataFrame(results)
    best = df.sort_values(by="distance").head(top_n)
    print("Meilleurs paramètres trouvés : ")
    print(best[["alpha", "beta", "decay", "distance"]].to_string(index=False))

    best_params = best.iloc[0][["alpha", "beta", "decay"]].to_dict()
    print("Meilleure combinaison : ")
    print(best_params)
explore_parameters()
show_best_parameters()

Removed 4 edges of 105
Costed 20 edges of 101
Removed 4 edges of 105
Costed 20 edges of 101
Removed 11 edges of 105
Costed 17 edges of 94
Removed 16 edges of 105
Costed 8 edges of 89
Removed 9 edges of 105
Costed 15 edges of 96
Removed 15 edges of 105
Costed 18 edges of 90
Removed 15 edges of 105
Removed 14 edges of 105
Costed 6 edges of 91
Removed 15 edges of 105
Costed 16 edges of 90
Removed 18 edges of 105
Costed 17 edges of 87
Removed 2 edges of 105
Costed 19 edges of 103
Removed 8 edges of 105
Costed 6 edges of 97
Removed 5 edges of 105
Costed 5 edges of 100
Removed 8 edges of 105
Costed 0 edges of 97
Removed 2 edges of 105
Removed 0 edges of 105
Costed 8 edges of 105
Removed 15 edges of 105
Costed 17 edges of 90
Costed 3 edges of 105
Removed 16 edges of 105
Costed 10 edges of 89
Removed 9 edges of 105
Costed 15 edges of 96
Removed 9 edges of 105
Costed 11 edges of 96
Removed 3 edges of 105
Costed 4 edges of 102
Removed 7 edges of 105
Costed 6 edges of 98
Removed 14 edges of 105
C

on verif avec le mm graph

In [2]:
test_graph = RandomGraph(node_number=15, min_weight=1, max_weight=100)
explore_parameters(test_graph)
show_best_parameters()

Removed 16 edges of 105
Costed 9 edges of 89
Removed 8 edges of 105
Costed 6 edges of 97
Removed 10 edges of 105
Removed 6 edges of 105
Removed 15 edges of 105
Costed 16 edges of 90
Removed 6 edges of 105
Costed 13 edges of 99
Removed 13 edges of 105
Removed 16 edges of 105
Costed 8 edges of 89
Removed 3 edges of 105
Costed 18 edges of 102
Removed 18 edges of 105
Costed 6 edges of 87
Removed 7 edges of 105
Costed 18 edges of 98
Removed 13 edges of 105
Costed 16 edges of 92
Removed 4 edges of 105
Costed 12 edges of 101
Removed 13 edges of 105
Costed 11 edges of 92
Removed 7 edges of 105
Costed 17 edges of 98
Removed 2 edges of 105
Costed 15 edges of 103
Removed 17 edges of 105
Costed 10 edges of 88
Removed 2 edges of 105
Costed 19 edges of 103
Removed 7 edges of 105
Costed 2 edges of 98
Removed 16 edges of 105
Costed 2 edges of 89
Removed 18 edges of 105
Costed 13 edges of 87
Removed 2 edges of 105
Costed 19 edges of 103
Removed 16 edges of 105
Costed 4 edges of 89
Removed 0 edges of 10

In [3]:
# def get_iteration_over_node_size(max_node_number = 100, threshold = 20, step = 5):
#     if(max_node_number < 6):
#         max_node_number=6
#     iterations = []
#     iteration_range = range(1,max_node_number + 2, step)
#     for i in iteration_range:
#         graph = RandomGraph(node_number=i,min_weight=1,max_weight=100)
#         ACO = GeneticAlgo(graph=graph,name="test", convergence_threshold = threshold)
#         ACO.run()
#         iterations.append(ACO.total_interations_realized)
#         print(f"{i} node step realized on {max_node_number}")
#     Plot.plot_iterations_over_node_sizes(iterations)

        

In [4]:
get_best_aco_parameters(graph, threshold=20, nodes_number=node_number, num_vehicles=1, test_iterations=test_iteration,execute_best_for_comparison=True, best_decay = 0.3, best_ants=500, best_alpha = 1.5, best_beta = 2)

NameError: name 'get_best_aco_parameters' is not defined